## 判断式
#!/bin/bash - #!/usr/bin/env bash 指定文件语法类型,bash  
### 利用test指令的测试功能
```bash
# test -e filename && echo "exist" || echo "Not exist"
```  
| 测试的标识 | 代表意义 | 是否常用 |
|:------|:-------|:------|
|1.关于某个文件名的文件类型判断|test -e filename||
|-e|是否存在|常用|
|-f|是否存在且为file|常用|
|-d|是否存在且为directory|常用|
|-b|是否存在且为block device||
|-c|是否存在且为character device||
|-S|是否存在且为Socket||
|-p|是否存在且为FIFO||
|-L|是否存在且为链接文件||
|2.关于文件的权限侦测|test -r filename||
|-r|可读||
|-w|可写||
|-x|可执行||
|-u|SUID||
|-g|SGID||
|-k|Sticky bit||
|-s|非空白文件||
|3.两个文件之间的比较|test file1 -nt file2||
|-nt|新于||
|-ot|旧于||
|-ef|同一文件 -- 硬链接||
|4.关于两个整数之间的判定|test n1 -eq n2||
|-eq|equal == ||
|-ne|not equal != ||
|-gt|greater than > ||
|-lt|less than < ||
|-ge|greater than or equal >= ||
|-le|less than or equal <= ||
|5.判断字符串的数据|||
|test -z string|是否为空串||
|test -n string|是否不为空串||
|test str1 == str2|是否相等||
|test str1 != str2|是否不等||
|6.多重条件判定|test -r filename -a -x filename||
|-a|and||
|-o|or||
|!|not||

### 利用判断符号 [ ]
```bash
#!/usr/bin/env bash

# [] 为bash 语法,执行需使用 bash *.sh
read -p "Please input (Y/N) : " yn
[ $yn == "Y" -o $yn == "y" ] && echo "OK, continue" && exit 0
[ $yn == "N" -o $yn == "n" ] && echo "Oh, interrupt" && exit 0
echo "I don't know what your choice is" && exit 0
```  
### Shell script的默认变量
```
scriptname opt1 opt2 opt3 opt4  
$0    $1    $2    $3    $4
$# -- 参数个数
$@ -- "$1 $2 $3 $4"
$* -- "$1 $2 $3 $4"
```  
```bash
#!/bin/bash

[ $# == 3 ] && echo 33 && exit 0
[ $# == 1 ] && echo 11 && exit 0
```

## 条件判断式
### 利用 if ... then
* 单层
```
if [ 条件判断式 ]; then
    ...
fi
```  
* && -- and
* || -- or
```
[ "${yn}" == "Y" -o "${yn}" == "y" ] 
==> 
[ "${yn}" == "Y" ] || [ "${yn}" == "y" ] 
```
* 多重/复杂条件判断式
```
if [ statement ]; then
    ...
else
    ...
fi
```  
---
```
if [ statement ]; then
    ...
elif [ statement ]; then
    ...
else
    ...
fi
```  
### 利用 case ... ... esac
```
case $变量名称 in
    "first")
        ...
        ;;
    "second")
        ...
        ;;
    *)
        ...
        exit 1
        ;;
esac
```  
---  
```bash
#!/bin/bash

case $1 in
    "hello")
	echo "Hello, how are you ?"
	;;
    "world")
	echo "See the world !"
	;;
    "")
	echo "You must input parameters, ex> {$0 someword}"
	;;
    *)
	echo "Usage $0 {hello}"
esac
```  
### 利用 function
```bash
#!/bin/bash

# check if has java & maven

install_java=`java -version 2>&1`

if [ -z "$install_java" ]; then
   echo "java is not install"
   exit 1
fi


function build()
{
    # check mvn installed
    install_mvn=`mvn -v 2>&1`
    if [ -z "$install_mvn" ]; then
        echo "maven is not install"
        exit 1
    fi
    mvn clean package -DskipTests
}

function get_pid()
{
    port=$1
    pid=$(lsof -i tcp:"$port"|grep 'LISTEN'|awk '{print $2}');
    if [ -n "$pid" ];then
        echo "$pid"
    else
        echo 0
    fi
}

function start()
{
    proj=$1
    profile=$2
    echo "run app $proj"
    cd "$cur"
    if test -f "target/$proj-0.0.1-SNAPSHOT.jar"; then
      chmod +x "target/$proj-0.0.1-SNAPSHOT.jar"
      if [ -z "$profile" ];then
          java -jar "target/$proj-0.0.1-SNAPSHOT.jar" --spring.profiles.active="dev" > "/tmp/logs/${proj}.log" 2>&1 &
      else
          java -jar "target/$proj-0.0.1-SNAPSHOT.jar" --spring.profiles.active="$profile" > "/tmp/logs/${proj}.log" 2>&1 &
      fi
    else
      echo "target/$proj-0.0.1-SNAPSHOT.jar not exists or not executable"
    fi
}

function stop()
{
    proj=$1
    port=$2
    pid=$(get_pid "$port")
    pid=${pid:-0}
    if [ "$pid" -gt 0 ];then
        echo "stop app $proj at port $port for pid $pid"
        kill -9 "$pid"
    else
        echo "app $proj is not running"
    fi
}

# online profile variable
app="***"
port=8480
profile=prod

case $1 in
    build)
        build
    ;;
    start)
        start "$app" "$profile"
    ;;
    stop)
        stop "$app" "$port"
    ;;
    restart)
        stop "$app" "$port"
        start "$app" "$profile"
    ;;
    *)
        echo "Usage: $0 [build|start|stop|restart]"
    ;;
esac
exit 0
```

## 循环(loop)
### while do done, until do done (不定循环)
```
while [ condition ]; do
    ...
done
```  
----
```
until [ condition ]; do
    ...
done
```  
计算 1+2+...+100 的结果  
```bash
#!/bin/bash

index=0
sum=0

while [ $index -le "100" ]; do
    sum=$(($sum+$index))
    index=$(($index+1))
done
echo $sum
```  
### for ... do ... done (固定循环)
```
for var in con1 con2 con3 ...; do
    ...
done
```  
批量上传文件  
```bash
#!/usr/bin/env bash

set -e

image_dist_path=images/

HOST_LIST='
10.19.119.187
10.19.12.220
10.19.167.2
10.19.170.88
10.19.19.248
10.19.16.116
10.19.52.143
10.19.117.87
'

for host in $HOST_LIST;do
    zip_path=$image_dist_path$host/images.zip
    dist_dir=/home/lab/disk02/publicData/zmoji_images/$host
    sshpass -p lab ssh lab@172.18.16.155 "
    	mkdir $dist_dir
    "
    sshpass -p lab scp $zip_path lab@172.18.16.155:$dist_dir
done
```  
### for ... do ... done 的数值处理
```bash
#!/bin/bash
sum=0
for ((i=0;i<=100;i=i+1));do
    sum=$(($sum+$i))
done
echo $sum
```  
----  
test - 计算生日剩余时间  
```bash
#!/bin/bash

read -p "Please input your birthday date (MMDD ex>0206) : " date2

date_d=$(echo $date2 | grep '[0-9]\{4\}')
# 双引号包裹变量,避免变量为空
if [ "$date_d" == "" ]; then
    echo "Wrong date"
    exit 1
fi
now=`date +%m%d`
if [ $date2 == $now ]; then
    echo "Happy birthday!"
elif [ $date2 -gt $now ]; then
    year=`date +%Y`
    total_d=$(($((`date --date=$year$date2 +%s` - `date +%s`))/60/60/24))
    echo "Your birthday will be $total_d later."
else 
    year=$((`date +%Y`+1))
    total_d=$(($((`date --date=$year$date2 +%s` - `date +%s`))/60/60/24))
   echo "Your birthday will be $total_d later."
fi
```